RTK QUERY

In [ ]:
// src/services/product.js

/* RTK Query
- Đặt tên file theo NAMESPACE
*/
// 1. import createApi
import { createApi, fetchBaseQuery } from "@reduxjs/toolkit/query/react";

export const productApi = createApi({
    reducerPath: "productApi",
    baseQuery: fetchBaseQuery({
        baseUrl: "https://api01.f8team.dev/api",
    }),
    endpoints: (builder) => ({
        getProducts: builder.query({ // use + getProducts + query
            query: () => '/products'
        })
    })
});

export const {useGetProductsQuery} = productApi

In [ ]:
// src/features/product/productSlice.js

import { createSlice } from "@reduxjs/toolkit";

const initialState = {
    products: [],
    value: 0,
};

const productSlice = createSlice({
    name: "product",
    initialState,
    reducers: {
        increase(state) {
            state.value++;
        },
    },
});

export const { increase } = productSlice.actions;
export default productSlice.reducer;


In [ ]:
//src/store.js

import { configureStore } from "@reduxjs/toolkit";
import productReducer from "./features/product/productSlice";
import { productApi } from "./services/product";

/* Đọc docs: https://redux-toolkit.js.org/tutorials/rtk-query */
const store = configureStore({
    reducer: {
        product: productReducer,
        [productApi.reducerPath]: productApi.reducer,
        //... --> Thêm nhiều ở đây
    },
    middleware: (getDefaultMiddleware) => [
        // getDefaultMiddleware().concat(productApi.middleware),
        ...getDefaultMiddleware(),
        productApi.middleware,
        //... --> Thêm nhiều ở đây
    ],
});

window.store = store;

export { store };


In [ ]:
//src/pages/Home/index.js

import { useGetProductsQuery } from "@/services/product";
import React from "react";

function Home() {
    // Xử lý loading
    const { isLoading, data } = useGetProductsQuery();
    /* Lưu ý:
        isFetching: Đang fetch dữ liệu
        isLoading: Cũng đang fetch nhưng chưa nhận được dữ liệu
        isSuccess: Nhận dữ liệu thành công chưa
    */

    return (
        <div>
            <h1>Product List</h1>
            <ul>
                {isLoading ? (
                    <div>Loading</div>
                ) : (
                    data.data.items.map((product) => (
                        <li key={product.id}>
                            {product.id}.{product.title}
                        </li>
                    ))
                )}
            </ul>
        </div>
    );
}

export default Home;


In [ ]:
// src/services/baseQuery.js

import { fetchBaseQuery } from "@reduxjs/toolkit/query";

const baseQuery = fetchBaseQuery({
    baseUrl: "https://api01.f8team.dev/api",
});

export default baseQuery


In [ ]:


import { createApi } from "@reduxjs/toolkit/query/react";
import baseQuery from "./baseQuery"; // Import baseQuery

export const productApi = createApi({
    reducerPath: "productApi",
    baseQuery, // Sửa ở đây
    endpoints: (builder) => ({
        getProducts: builder.query({ // query: truy vấn/lấy ra => get
            query: () => "/products",
        }),
    }),
});

export const { useGetProductsQuery } = productApi;


In [ ]:
// Sửa baseQuery

// import { fetchBaseQuery } from "@reduxjs/toolkit/query";

import httpRequest from "@/utils/httpRequest";

// const baseQuery = fetchBaseQuery({
//     baseUrl: "https://api01.f8team.dev/api",
// });

const baseQuery = async (url, { signal, dispatch, getState }, extraOptions) => {
    console.log(url, signal, dispatch, getState, extraOptions);
    try {
        const data = await httpRequest({
            method: "GET",
            url,
        });
        console.log(data);
        return { data: data }; //response trả về từ API
    } catch (error) {
        return error;
    }
};

/* 
- arg (url) --> /products (endpoints)
- signal --> cancel request: Giả sử request đang gửi, mà bạn thoát khỏi màn hình, không cần response nữa => viết vào cleanup để cancel request

*/
export default baseQuery;

In [ ]:
// Với method: POST

// src/services/product.js


import { createApi } from "@reduxjs/toolkit/query/react";
import baseQuery from "./baseQuery";

export const productApi = createApi({
    reducerPath: "productApi",
    baseQuery,
    endpoints: (builder) => ({
        getProducts: builder.query({
            query: () => "/products",
        }),
        // Thêm phương thức tạo product
        createProduct: builder.mutation({ // Những hành động: Thêm/sửa/xoá => mutation
            query: (body) => ({
                url: "/products",
                method: "POST",
                body,
                // Bổ sung headers
                headers: {
                    Authorization: "Bearer <token>",
                },
            })
        })
    }),
});

// Bổ sung useCreateProductMutation
export const { useGetProductsQuery, useCreateProductMutation } = productApi;


In [ ]:
//src/services/baseQuery.js

import httpRequest from "@/utils/httpRequest";

const baseQuery = async (
    args,
    { signal, dispatch, getState },
    extraOptions,
) => {
    //...

    const isObject = typeof args === "object";

    /* Kiểm tra nếu arg là object hay là chuỗi */
    const config = {
        url: isObject ? args.url : args,
        method: isObject ? args.method : "GET",
    };

    /* Kiểm tra xem args có body, headers gửi đi không, nếu có thì thêm vào config */
    if (isObject) {
        if (args.body) config.data = args.body;
        if (args.headers) config.headers = args.headers;
    }

    
     try {
        const response = await httpRequest(config);

        return {
            data: response,
        };
    } catch (error) {
        return error;
    }
};

//...


In [ ]:
// src/pages/Home/index.jsx

//...

function Home() {
    const { isLoading, data } = useGetProductsQuery();
    // Lấy ra từ RTK
    const [createProduct, newProductResponse] = useCreateProductMutation();
    console.log(createProduct); 

    const handleCreateProduct = () => {
        // Đối số args khi gọi hàm là để truyền vào body trong base Query
        createProduct({ title: "New Product" });
    };

    return (
        <div>
            <button onClick={handleCreateProduct}>Create New Product</button>

            <h1>Product List</h1>
            <ul>
                {isLoading ? (
                    <div>Loading</div>
                ) : (
                    data.data.items.map((product) => (
                        <li key={product.id}>
                            {product.id}.{product.title}
                        </li>
                    ))
                )}
            </ul>
        </div>
    );
}

export default Home;
